In [1]:
# importing library
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 64
batch_size = 128
learning_rate = 3e-4
max_iters = 3000
eval_iters = 250
n_embd = 384              # length of embedding vector
n_layer= 8                # determine number of decoder or encoder
n_head = 8                # 4 heads = 4 decoder 
dropout = 0.2             # dropping 20% neuron

cuda


In [2]:
# This method is not suitable for large corpus of data like openweb text so we need some data preprocessing
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))           # provide sorted set of all characters present in wizard_of_oz.txt
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
# character tokenizer : less vocab
string_to_int = { ch:i for i,ch in enumerate(chars)}      # encoding each character from 0 to len(chars) e.g 'A' : 0
int_to_string = {i:ch for i,ch in enumerate(chars)}       # encoding each index from starting character to end 0 : 'A'
encode = lambda s: [string_to_int[c] for c in s]          # lamda function : takes string and encode each char to integer
decode = lambda l: ''.join([int_to_string[i] for i in l]) # takes a list of integer and encode it to char, form string using .join()

In [4]:
encoded_hello = encode('hello')
encoded_hello

[61, 58, 65, 65, 68]

In [5]:
decoded_number = decode(encoded_hello)
decoded_number

'hello'

In [6]:
# using pytoch
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [7]:
# creating training and test set
n= int(0.8*len(text))                        # 80%  of text
train_data = data[:n]                        # start to 80%
val_data = data[n:]                              # 80% to 100% : 20% as validation

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))        # select the random starting point for subsequences
    x = torch.stack([data[i:i+block_size] for i in ix])              # select the input of block size randomly and stack
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])          # select the target of block size randomly and stack
    x,y = x.to(device), y.to(device)
    return x,y                                                      # x: e.g from 0to5 , y: e.g from 1 to 6  so its like  predicting a next character                                             
x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[75, 58, 57,  ..., 59,  1, 56],
        [65, 78,  1,  ..., 68, 72, 73],
        [74, 73,  1,  ..., 68, 76, 58],
        ...,
        [ 0,  3, 39,  ..., 35, 54, 67],
        [74, 73,  1,  ..., 54, 72, 58],
        [ 1, 73, 61,  ...,  0,  0,  3]], device='cuda:0')
targets:
tensor([[58, 57,  1,  ...,  1, 56, 68],
        [78,  1, 67,  ..., 72, 73,  1],
        [73,  1, 68,  ..., 76, 58, 57],
        ...,
        [ 3, 39, 67,  ..., 54, 67, 72],
        [73,  1, 67,  ..., 72, 58,  1],
        [73, 61, 54,  ...,  0,  3, 47]], device='cuda:0')


In [8]:
#@ processing sequential data
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [9]:
#@ Implementaton of GPT language model
class Head(nn.Module):
    """ One head of self attention """
    
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))      # for masking multi head
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # input of size (batch, timestep, channels)
        # output of size (batch, timestep, head_size)
        B,T,C = x.shape                        # unpacking the shape
        k = self.key(x)                        # B,T,hs
        q = self.query(x)                      # B,T,hs
        # compute attention scores
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5                            # (B,T,hs) @ (B,hs,T) = (B,T,T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))                # (B,T,T) : mask and form lower triangular matrix
        wei = F.softmax(wei, dim=-1)                                                # apply softmax block wise and highlight the value
        wei = self.dropout(wei)                                                     
        # perform the weighted aggregation of the values
        v = self.value(x)                         # B,T,hs
        out =  wei @ v                            # (B,T,T) @ (B,T,hs) = (B,T,hs)
        return out
    
    
class MultiHeadAttention(nn.Module):                                                # Multihead attention : Determine attention for multiple head
    def __init__(self,num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])           
        self.proj = nn.Linear(head_size * num_heads, n_embd )
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads], dim = -1)            # (B:batch,T:time,C:feature) : [h1,h1,h1,h1,h2,h2,h2,h2.....,hn,hn,hn,hn]
        out = self.dropout(self.proj(out))
        return out
    
class FeedForward(nn.Module):               # feedforward of decoder 
    def __init__(self, n_embd):             # linear -> RELU -> Linear
        super().__init__()
        self.net = nn.Sequential(
        nn.Linear(n_embd, 4*n_embd),
        nn.ReLU(),
        nn.Linear(4 * n_embd, n_embd),
        nn.Dropout(dropout),)
    def forward(self,x):
        return self.net(x)
        
class Block(nn.Module):
    #Transformer block : communication followed by computation
    
    def __init__(self,n_embd,n_head):  # constructor initializing decoder member item
        #n_embd : embedding dimension , n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head                    # number of feature each head is caputring
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self,x):            # forming deocder block         
        y = self.sa(x)              # multihead attention
        x = self.ln1(x+y)           # Adding and norm
        y = self.ffwd(x)            # feedforward : Linear -> RELU -> Linear
        x = self.ln2(x + y)         # Adding and norm
        return x
    

class GPTLanguageModel(nn.Module):
    
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)           # token embedding 
        self.position_embedding_table = nn.Embedding(block_size, n_embd)        # positional embedding 
        
        # decoder 
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for i in range(n_layer)])  # providing  decoder = n_layer sequentially
        
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)  
        self.apply(self._init_weights)
        
        
    def _init_weights(self, module):                                            # weight initialization
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)                               # set bias to zeros
        elif isinstance(module, nn.Embedding):                                  # if module is object of nn.Embedding 
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)            # normal initialization of weights with less variation
    
    
    def forward(self, index, targets = None):
        B,T = index.shape
        # index and targets are both (B,T) tensor of integers
        tok_embd = self.token_embedding_table(index)       # (B,T,C)
        pos_embd = self.position_embedding_table(torch.arange(T, device=device))  # (T,C)
        x= tok_embd + pos_embd                                                    # (B,T,C ) by broadcasting
        x= self.blocks(x)                                                         # decoder 
        x= self.ln_f(x)                                                           # layer normalizing
        logits= self.lm_head(x)                                                   # (B,T,vocab_size)
                                                                                   
        if targets is None:
            loss = None
        else:
            B, T, C  = logits.shape                       # B (batch size), T (sequence length), and C (embedding dimensions)
            logits = logits.view(B*T, C)                  # reshape to total word, embedding dimension
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)       # loss between predicted logits and actual logits
        return logits, loss                               # return predicted logits and loss
    
    def generate(self , index, max_new_tokens):
        # index is (B,t) array of indices in the current context
        for i in range(max_new_tokens):
            logits, loss = self.forward(index)            # get the predictions
            # focus on the last step only
            logits = logits[:,-1,:]                       # becomes (B,C)
            # apply softmax to get prob.
            probs = F.softmax(logits, dim=-1)             # (B,C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples =1) # (B,1)
            # append sampled inext to the running sequene
            index = torch.cat((index, index_next), dim=1)        # (B, T+1)
        return index
    
model = GPTLanguageModel(vocab_size)
m = model.to(device)

In [10]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits,loss = model(X,Y)
            losses[k] = loss.item()
        out[split]  = losses.mean()
    model.train()
    return out

In [11]:
# create a pytorch optimizer
optimizer =  torch.optim.AdamW(model.parameters(),lr=learning_rate)            # AdamW as optimizer
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss : {losses['train']:.3f}, val_loss: {losses['val']:.3f} ")
    xb, yb = get_batch('train')                                                    # sample a batch of data
    
    logits,loss = model.forward(xb,yb)                                             # forward props
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss : 4.468, val_loss: 4.463 
step: 250, train loss : 1.955, val_loss: 2.059 
step: 500, train loss : 1.791, val_loss: 1.915 
step: 750, train loss : 1.721, val_loss: 1.872 
step: 1000, train loss : 1.675, val_loss: 1.824 
step: 1250, train loss : 1.640, val_loss: 1.816 
step: 1500, train loss : 1.611, val_loss: 1.783 
step: 1750, train loss : 1.585, val_loss: 1.766 
step: 2000, train loss : 1.568, val_loss: 1.746 
step: 2250, train loss : 1.549, val_loss: 1.742 
step: 2500, train loss : 1.538, val_loss: 1.734 
step: 2750, train loss : 1.529, val_loss: 1.728 
1.5951141119003296


In [17]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(context)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)
    

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
